In [2]:
import numpy as np 
import xarray as xr
from math import sqrt
from scipy.spatial import cKDTree

In [3]:
def rmse(observed, modeled):
    #classic rmse
    observed = observed.flatten()
    modeled = modeled.flatten()
    return np.sqrt(((observed - modeled) ** 2).mean())


def rmse_ref(observed, modeled):
    #for fss denominator
    total_gridcells = len(observed.flatten())
    return np.sqrt((np.nansum(observed**2) + np.nansum(modeled**2))/total_gridcells)


def mse(observed, modeled):
    #classic rmse
    observed = observed.flatten()
    modeled = modeled.flatten()
    return ((observed - modeled) ** 2).mean()


def mse_ref(observed, modeled):
    #for fss denominator
    total_gridcells = len(observed.flatten())
    return (np.nansum(observed**2) + np.nansum(modeled**2))/total_gridcells


def compute_sns(ob_file, model_file):
        
    d_record = 1 - (rmse(ob_file, model_file)/rmse_ref(ob_file, model_file))
    d2_record = 1 - (mse(ob_file, model_file)/mse_ref(ob_file, model_file))
        
    return d_record, d2_record

In [6]:
def computing_errors(wrf_file_folder_choice, UH_threshold, dbz_threshold, binary=True):
    
    """
    Function to compute sns, rmse, and mse and save the final output as a netCDF file.
    """
   
    def hypot(a,b):
        """
        Compute hypotenuse for radius of interest in regridding step for sns.
        """
        return sqrt(a**2 + b**2)

    data = xr.open_dataset('/glade/work/molina/DATA/jan2017_synoptic/'+wrf_file_folder_choice+'/ver_grid_20_UH_'+str(UH_threshold)+'_Z_'+str(dbz_threshold)+'.nc')
  
    data_proxy = data.UHdbz_mask
    data_tors = data.tors_mask

    stagger = np.arange(1,140,1)

    thesns_rmse = np.zeros((len(stagger)))
    thesns_mse = np.zeros((len(stagger)))
    thermse = np.zeros((len(stagger)))
    themse = np.zeros((len(stagger)))

    i_grid, j_grid = np.indices(data_proxy[:,:].shape[:])

    #grab the values
    max_proxy = data_proxy
    #grab values where the threshold is exceeded
    max_points_proxy = np.array(np.where(max_proxy >= 1)).T
    #create kdtree using max points
    max_tree_proxy = cKDTree(max_points_proxy)

    #grab the values
    max_tors = data_tors
    #grab values where the threshold is exceeded
    max_points_tors = np.array(np.where(max_tors >= 1)).T
    #create kdtree using max points
    max_tree_tors = cKDTree(max_points_tors)

    binary = True

    for num, stag in enumerate(stagger):

        #create the new grid
        stagger_points_proxy = np.vstack((i_grid[::stag, ::stag].ravel(), j_grid[::stag, ::stag].ravel())).T
        stagger_points_tors = np.vstack((i_grid[::stag, ::stag].ravel(), j_grid[::stag, ::stag].ravel())).T
        valid_stagger_points_proxy = np.zeros(stagger_points_proxy.shape[0])
        valid_stagger_points_tors = np.zeros(stagger_points_tors.shape[0])
        stagger_tree_proxy = cKDTree(stagger_points_proxy)
        stagger_tree_tors = cKDTree(stagger_points_tors)

        if len(max_points_proxy) != 0:
            hit_points_proxy = np.unique(np.concatenate(max_tree_proxy.query_ball_tree(stagger_tree_proxy, hypot(stag,stag))))
            hit_points_tors = np.unique(np.concatenate(max_tree_tors.query_ball_tree(stagger_tree_tors, hypot(stag,stag))))
            if not binary:
                valid_stagger_points_proxy[hit_points_proxy.astype(int)] += 1
                valid_stagger_points_tors[hit_points_tors.astype(int)] += 1
                surrogate_grid_proxy = valid_stagger_points_proxy.reshape(i_grid[::stag, ::stag].shape) 
                surrogate_grid_tors = valid_stagger_points_tors.reshape(i_grid[::stag, ::stag].shape) 
            if binary:
                valid_stagger_points_proxy[hit_points_proxy.astype(int)] = 1
                valid_stagger_points_tors[hit_points_tors.astype(int)] = 1
                surrogate_grid_proxy = valid_stagger_points_proxy.reshape(i_grid[::stag, ::stag].shape) 
                surrogate_grid_tors = valid_stagger_points_tors.reshape(i_grid[::stag, ::stag].shape) 

        thesns_rmse[num], thesns_mse[num] = compute_sns(valid_stagger_points_proxy, valid_stagger_points_tors)
        thermse[num] = rmse(valid_stagger_points_proxy, valid_stagger_points_tors)
        themse[num] = mse(valid_stagger_points_proxy, valid_stagger_points_tors)
    
    thedata = xr.Dataset({'sns_rmse':(['x'],thesns_rmse),
                          'sns_mse':(['x'],thesns_mse),
                          'rmse':(['x'],thesns_rmse),
                          'mse':(['x'],thesns_rmse)},
                        attrs={'File Author':'Maria J. Molina'})

    thedata.to_netcdf('/glade/work/molina/DATA/jan2017_synoptic/'+wrf_file_folder_choice+'/errors_file_UH_'+str(UH_threshold)+'_Z_'+str(dbz_threshold)+'.nc')

    return

In [12]:
#wrf_file_folder_choice = 'wrf4km_ens_1'
#wrf_file_folder_choice = 'wrf4km_ens_2'
#wrf_file_folder_choice = 'wrf4km_ens_3'
#wrf_file_folder_choice = 'wrf4km_ens_4'
#wrf_file_folder_choice = 'wrf4km_ens_5'
wrf_file_folder_choice = 'wrf4km_ens_6'

UH_thresholds = np.array([100,100,100,80,80,80,60,60,60,40,40,40])
dbz_thresholds = np.array([50,40,30,50,40,30,50,40,30,50,40,30])

for UH_threshold, dbz_threshold in zip(UH_thresholds, dbz_thresholds):
    print(f"Computing UH:{UH_threshold} and dbz:{dbz_threshold}")
    computing_errors(wrf_file_folder_choice, UH_threshold, dbz_threshold, binary=True)

Computing UH:100 and dbz:50
Computing UH:100 and dbz:40
Computing UH:100 and dbz:30
Computing UH:80 and dbz:50
Computing UH:80 and dbz:40
Computing UH:80 and dbz:30
Computing UH:60 and dbz:50
Computing UH:60 and dbz:40
Computing UH:60 and dbz:30
Computing UH:40 and dbz:50
Computing UH:40 and dbz:40
Computing UH:40 and dbz:30
